In [ ]:
pip install sentence-transformers numpy


In [ ]:
DOCUMENT_TEXT = """
Executive Summary:
2024 marked a year of both opportunity and challenge for FinSolve Technologies. Despite a robust revenue increase, we saw significant pressure in certain expense categories, notably vendor-related costs and software subscriptions. However, these pressures were balanced by cost-saving measures in operational efficiency, strong gross margin performance, and strategic investment in growth areas. The company is well-positioned to continue scaling its core offerings, but focused attention on cost optimization will be essential for maintaining profitability in the coming years.

Year-Over-Year (YoY) Analysis:
FinSolve Technologies's revenue grew by 25% in 2024, driven largely by the global expansion of its services, especially in Asia and Europe. This was accompanied by a 10% increase in vendor-related expenses, impacting overall profit margins. While gross profit increased by 25%, reflecting higher operational efficiency, net income saw a more modest increase of 12%. This suggests that while revenue growth is strong, controlling vendor costs and maintaining healthy cash flows remain key to long-term profitability.

Expense Breakdown by Category:
The primary drivers of expense in 2024 were:

Vendor Services - A total of $30M, representing a 18% increase from the previous year. The largest contributors were:

Marketing-related expenses (Dinner, corporate events) – 40% of vendor services.
Training and education expenses – 30% of vendor services.
Software subscriptions (cloud services, licensing) – 25% of vendor services.
Other miscellaneous expenses – 5% of vendor services.
Analysis: The Dinner and Training categories accounted for an increasing share of the marketing budget. While essential for brand positioning, these expenses need tighter management, potentially through vendor renegotiations or reduced event frequency.

Software Subscriptions - A significant expense totaling $25M, up 22% from 2023. Given the heavy reliance on cloud-based tools and SaaS subscriptions, this area could benefit from more rigorous contract negotiation and potential consolidation of service providers.

Employee Benefits and HR Costs - With FinSolve Technologies's growth in headcount, HR expenses (benefits, recruitment, training) saw a 10% increase. While employee growth is essential, optimizing benefits packages and hiring processes could reduce per-employee cost.

Other Operational Expenses - A mix of general operational and administrative expenses totaling $15M, with a notable increase in travel and miscellaneous office costs, which grew by 8% year-over-year.

Cash Flow Analysis:
FinSolve Technologies’s cash flow from operations remained positive, amounting to $50M, a 20% increase over the prior year. However, the delayed payment cycles from several key vendors resulted in accounts payable delays, which slightly impacted cash liquidity during the second half of 2024. Addressing these delays, particularly in vendor payments, will be crucial to ensuring smoother cash flow management.

Cash Flow from Investing Activities: Investments in research and development, new market entry, and acquisitions amounted to $15M, representing 20% of total cash flows. While these investments are crucial for long-term growth, they reduced short-term liquidity, which is something that should be monitored closely.

Cash Flow from Financing Activities: Financing activities remained consistent, with a small net cash inflow of $5M, used primarily for working capital management and capital expenditure.

Key Financial Ratios and Metrics:
Gross Margin: 60% (Up from 55% in 2023)

Industry Benchmark: 55% (Indicates FinSolve Technologies is performing above average in terms of operational efficiency and cost control relative to peers).
Net Margin: 12% (Consistent with 2023)

Industry Benchmark: 10%
FinSolve Technologies’s stable net margin indicates sound control over both direct and indirect costs, though further improvement is possible with better vendor management.
Return on Investment (ROI): 15%

Industry Benchmark: 12%
This figure shows efficient use of invested capital, outpacing the industry, and is a positive indicator for investor confidence.
Operating Expenses to Revenue Ratio: 40%

Industry Benchmark: 45%
A lower operating expense ratio than industry peers suggests strong operational discipline.
Days Sales Outstanding (DSO): 45 days

Industry Benchmark: 30 days
A high DSO indicates that receivables collection could be improved. Optimizing payment cycles will be key for cash flow stability.
Operational Insights:
In 2024, operational efficiency was significantly impacted by delays in payment cycles and rising vendor costs. The company achieved strong revenue growth, but it’s clear that managing vendor expenses and tightening accounts payable and receivable processes will be crucial for future profitability.

Recommendations:

Vendor Management: Continue negotiations for better pricing, especially in high-expense categories like marketing events and software subscriptions. Consider consolidating suppliers where possible to leverage better terms.
Automation in Vendor Payments: Automating the accounts payable process will reduce human errors, speed up payments, and help reduce overdue payments, which will positively impact cash flow.
Employee Cost Optimization: As the headcount grows, consider introducing more flexible benefits packages and using technology to reduce HR administrative costs.
Cost-Cutting in Non-Core Activities: Explore opportunities to reduce expenses in non-essential categories, including travel and certain office-related costs.
Risk Analysis and Mitigation Strategies:
Risk: Vendor Cost Inflation – Vendor services, particularly marketing and software subscriptions, have shown significant price inflation. To mitigate this, a comprehensive vendor risk management plan should be developed, ensuring that contracts are regularly reviewed, and prices negotiated.

Risk: Cash Flow Volatility – Delays in receivables and vendor payment cycles may lead to short-term cash flow volatility. Improved working capital management and streamlining payment processes can significantly reduce this risk.

Risk: Operational Scalability – As FinSolve Technologies scales, maintaining operational efficiency will become more challenging. This can be mitigated by investing in automation, streamlining workflows, and maintaining a leaner organizational structure.

Risk: Regulatory Compliance – As the company expands globally, navigating complex regulatory landscapes in different regions may introduce compliance risks. FinSolve Technologies must invest in legal and compliance teams to ensure adherence to local laws and regulations.

Appendices:
Detailed tables and charts are available in the financial statements section of the main report, showcasing YoY comparisons, expense breakdowns, and key ratios.
"""


In [ ]:
RBAC = {
    "admin": {
        "inherits": ["finance", "auditor"],
        "permissions": ["view_all"]
    },
    "finance": {
        "inherits": ["auditor"],
        "permissions": ["view_financial"]
    },
    "auditor": {
        "inherits": [],
        "permissions": ["view_limited"]
    }
}


In [ ]:
def expand_roles(role):
    expanded = set()
    stack = [role]

    while stack:
        r = stack.pop()
        if r not in expanded:
            expanded.add(r)
            stack.extend(RBAC[r]["inherits"])

    return expanded


In [ ]:
def chunk_text(text, chunk_size=120, overlap=30):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap

    return chunks


In [ ]:
def attach_metadata(chunks):
    chunk_objects = []

    for i, chunk in enumerate(chunks):
        if "Vendor" in chunk or "expenses" in chunk:
            access = "view_financial"
        else:
            access = "view_limited"

        chunk_objects.append({
            "id": f"chunk_{i}",
            "text": chunk,
            "metadata": {"access": access},
            "embedding": None
        })

    return chunk_objects


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_chunks(chunks):
    texts = [c["text"] for c in chunks]
    embeddings = model.encode(texts)

    for c, emb in zip(chunks, embeddings):
        c["embedding"] = emb

    return chunks


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def filter_by_permission(chunks, permissions):
    return [
        c for c in chunks
        if c["metadata"]["access"] in permissions
    ]


In [ ]:
def knn_search(query, chunks, top_k=3):
    query_emb = model.encode([query])[0]

    scored = []
    for c in chunks:
        score = np.dot(query_emb, c["embedding"])
        scored.append((score, c))

    scored.sort(reverse=True, key=lambda x: x[0])
    return [c for _, c in scored[:top_k]]


In [ ]:
def rbac_rag_pipeline(query, user_role):

    roles = expand_roles(user_role)

    permissions = set()
    for r in roles:
        permissions.update(RBAC[r]["permissions"])

    chunks = chunk_text(DOCUMENT_TEXT)

    chunk_objects = attach_metadata(chunks)

    chunk_objects = embed_chunks(chunk_objects)

    allowed_chunks = filter_by_permission(chunk_objects, permissions)

    results = knn_search(query, allowed_chunks)

    return results


In [ ]:
if __name__ == "__main__":
    user_query = "What happened to vendor expenses in 2024?"
    user_role = "finance"

    results = rbac_rag_pipeline(user_query, user_role)

    print("\nTop Results:\n")
    for r in results:
        print("-", r["text"][:200], "\n")



Top Results:

- 2024, driven largely by the global expansion of its services, especially in Asia and Europe. This was accompanied by a 10% increase in vendor-related expenses, impacting overall profit margins. While  ...

- 18% increase from the previous year. The largest contributors were: Marketing-related expenses (Dinner, corporate events) – 40% of vendor services. Training and education expenses – 30% of vendor serv ...

- Executive Summary: 2024 marked a year of both opportunity and challenge for FinSolve Technologies. Despite a robust revenue increase, we saw significant pressure in certain expense categories, notably ...

